# Разработка рекомендательного алгоритма для читателей библиотеки

## Описание задачи

В 2022 году ряд ведущих российских издательств, книжных ресурсов и библиотек отмечают рост спроса на электронные и бумажные книги. Фонды региональных и федеральных библиотек насчитывают миллионы экземпляров книг. Российская государственная библиотека (РГБ) — крупнейшая библиотека в нашей стране, предоставляющая современные цифровые сервисы, такие как электронная библиотека РГБ и Национальная электронная библиотека, благодаря которым читатели могут искать и просматривать книги, не выходя из дома.

Сейчас в фондах РГБ хранится более 47 миллионов книг и различных артефактов. Такой объем документов имеет огромную культурную, историческую и научную ценность, однако затрудняет процесс каталогизации. Поиск конкретных изданий и тематических подборок занимает время. Кроме непосредственного доступа до содержимого электронной книги, возникает потребность выполнять поиск по семантике книги и формировать ассоциативные связи между различными документами для того, чтобы в дальнейшем предлагать читателям более релевантный результат поиска, а также персонифицированные рекомендации.

Читатели библиотеки — это и любители художественных произведений, и специалисты, интересующиеся отраслевой литературой, ученые и соискатели, работающие в научных проектах. Разнообразие целевой аудитории усложняет разработку рекомендательной системы из-за высокой семантической сложности изданий.

В РГБ активно ведется процесс оцифровки документов, что делает возможным внедрение адаптивной системы поиска. В рамках чемпионата участникам предлагается разработать рекомендательный алгоритм для читателей библиотеки, который позволит осуществлять семантический поиск литературы и рекомендовать книги читателю на основе его персональных предпочтений. Такие подборки позволят посетителям библиотеки открыть для себя новые жанры, авторов и произведения, которые ранее им были неизвестны.

### Установка и импорт библиотек

In [1]:
# !pip install rectools

In [2]:
import pandas as pd
from implicit.nearest_neighbours import BM25Recommender
from rectools.dataset import Dataset
from rectools.models import ImplicitItemKNNWrapperModel, PopularModel

C:\Users\PronichevAV\AppData\Roaming\Python\Python38\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


### Загрузка данных

In [3]:
users = pd.read_csv('../data/users.csv', sep=';', index_col=None, dtype={'age': str, 'chb': str, 'chit_type': str, 'gender': str})
items = pd.read_csv('../data/items.csv', sep=';', index_col=None, dtype={'author': str, 'bbk': str, 'izd': str, 'sys_numb': str, 'title': str, 'year_izd': str})
train_transactions = pd.read_csv('../data/train_transactions_extended.csv', sep=';', index_col=None, dtype={'chb': str, 'date_1': str, 'is_printed': str, 'is_real': str, 'source': str, 'sys_numb': str, 'type': str})
sample_submission = pd.read_csv('../data/sample_solution.csv', sep=';', index_col=None)

### Предобработка данных

In [4]:
def get_age_bins(age):
    """Функция, распределяющая пользователей по возрастным категориям"""
    try:
        age = int(age)
        if 0 <= age <= 17:
            return 'age_0_17'
        elif 18 <= age <= 24: 
            return 'age_18_24'
        elif 25 <= age <= 34: 
            return 'age_25_34'
        elif 35 <= age <= 44: 
            return 'age_35_44'
        elif 45 <= age <= 54: 
            return 'age_45_64'
        elif age >= 65: 
            return 'age_65_inf'
    except:
        return None

Предобработка датасета взаимодействий

In [5]:
train_transactions['date_1'] = pd.to_datetime(train_transactions['date_1'], yearfirst=True)                       # преобразование в datetime
train_transactions.rename(columns={'chb': 'user_id', 'sys_numb': 'item_id', 'date_1': 'datetime'}, inplace=True)  # переименование столбцов
train_transactions['weight'] = 10                                                                                 # присвоение веса взаимодействиям
train_transactions

,user_id,item_id,datetime,is_real,type,source,is_printed,weight
0,100000641403,RSL01004206702,2021-02-21,yes,скачивание,dlib.rsl.ru,False,10
1,100000641403,RSL01000769304,2021-03-23,yes,скачивание,dlib.rsl.ru,False,10
2,100000641403,RSL01004211574,2021-02-21,yes,скачивание,dlib.rsl.ru,False,10
3,100000644359,RSL01009800093,2021-03-16,yes,книговыдача,единый просмоторщик,False,10
4,100000644359,RSL01003557352,2021-03-10,yes,книговыдача,единый просмоторщик,False,10
...,...,...,...,...,...,...,...,...
259561,300001173062,RSL01002975109,2022-03-10,yes,скачка,единый просмоторщик,False,10
259562,300001173062,RSL01002975109,2022-03-10,yes,книговыдача,единый просмоторщик,False,10
259563,400001035059,RSL01002298169,2021-07-01,yes,книговыдача,2DL.Viewer,False,10
259564,400001035059,RSL01002632325,2021-07-01,yes,книговыдача,2DL.Viewer,False,10


Предобработка датасета пользователей

In [6]:
users.rename(columns={'chb':'user_id'}, inplace=True)                                                           # переименование столбцов
users['age'] = users['age'].apply(get_age_bins)                                                                 # распределение по возрастным категориям
users = users[['user_id', 'age', 'gender']].copy()
users = users.loc[users["user_id"].isin(train_transactions["user_id"])].copy()                                  # удаление пользователей без взаимодействий
users['age'] = users['age'].fillna('unknown_age')                                                               # заполнение пропусков в возрасте 
users['gender'] = users['gender'].apply(lambda x: 'unknown_gender' if x in ['не указан', 'отсутствует'] else x) # заполнение пропусков в признаке "пол"

In [7]:
# Преобразование признаков пользователей в формат, совместимый с библиотекой rectools
user_features_frames = []
for feature in ["gender", "age", "chit_type"]:
    feature_frame = users.reindex(columns=["user_id", feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features = user_features.loc[~user_features['value'].isin(['не указан', 'отсутствует', 'нет данных']), :]
user_features.reset_index(drop=True, inplace=True)
user_features.head()

,id,value,feature
0,300001020830,female,gender
1,300001113642,female,gender
2,300001148466,female,gender
3,300001117011,female,gender
4,200001038094,female,gender


Предобработка датасета элементов

In [8]:
items.rename(columns={'sys_numb':'item_id'}, inplace=True)                                                 # переименование столбцов
items = items.loc[items["item_id"].isin(train_transactions["item_id"]), ['item_id', 'title']].copy()       # удаление элементов без взаимодействий

Создание датасета в формате, совместимом с библиотекой rectools

In [9]:
dataset = Dataset.construct(interactions_df=train_transactions,
                            user_features_df=user_features,
                            cat_user_features=['gender', 'age']
                            )

### Этап моделирования 

Модель включает в себя два этапа предсказаний - для теплых (у которых были взаимодействия) и холодных пользователей:

- На первом этапе применяется алгоритм ранжирования BM25. 

- На втором этапе - для холодных пользователей рекомендации создаются на основе популярных элементов.

Обучение и получение рекомендаций моделью первого этапа

In [10]:
%%time
model = ImplicitItemKNNWrapperModel(model=BM25Recommender(K=100, K1=0.05, B=0.1, num_threads=-1))
model.fit(dataset)
warm_recos = model.recommend(
    users=train_transactions['user_id'].unique(),
    dataset=dataset,
    k=20,
    filter_viewed=True,
)

Wall time: 1.58 s


Получение списка холодных пользователей

In [11]:
cold_users = list(warm_recos.groupby(by='user_id').sum().query('rank < 210').index) \
              + list(set(dataset.user_id_map.external_ids) - set(warm_recos['user_id']))

In [12]:
len(cold_users)

5990

Обучение и получение рекомендаций моделью второго этапа

In [13]:
%%time
model = PopularModel()
model.fit(dataset)
cold_recos = model.recommend(
    users=cold_users,
    dataset=dataset,
    k=20,
    filter_viewed=True,
)

Wall time: 170 ms


### Подготовка результата

In [14]:
full_recos = pd.concat([warm_recos[~warm_recos['user_id'].isin(cold_users)], cold_recos])

In [15]:
submission = full_recos.loc[:, ['user_id','item_id']]
submission.rename(columns={'user_id': 'chb', 'item_id': 'sys_numb'}, inplace=True)

In [16]:
submission.to_csv("../submissions/submission_from_nb.csv", index=False, sep=';')

In [17]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335060 entries, 0 to 119799
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   chb       335060 non-null  object
 1   sys_numb  335060 non-null  object
dtypes: object(2)
memory usage: 7.7+ MB
